## Set up and Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#!pip install --upgrade scipy numpy

In [3]:
print(np.__version__)

1.23.5


In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer

In [5]:
# load labels
train_label = np.load('train_label.npy')
test_label = np.load('test_label.npy')

In [6]:
train_label[:5]

array([34.20739619, 26.45371972, 34.96756055, 22.04476644, 25.84558824])

In [7]:
test_label[:5]

array([29.69849537, 30.84591837, 24.38979592, 36.25867883, 27.89129141])

In [8]:
# Load vgg16 features before relu: should have negative value
vgg16_train_features = np.load('vgg16_train_features.npy')
vgg16_test_features = np.load('vgg16_test_features.npy')

In [9]:
vgg16_train_features[:5]

array([[ -1.4681634 ,  -0.5275203 , -22.067028  , ..., -17.184917  ,
        -11.0921135 ,  -0.35512644],
       [ -0.20396854,  -0.71452796, -62.699623  , ..., -21.611687  ,
        -34.978123  ,  -0.25105998],
       [ -0.8742988 ,  -0.51126605, -30.323936  , ...,  -3.6527483 ,
         -9.731274  ,  -0.28503752],
       [ -0.71330154,  -0.4714042 , -33.486244  , ..., -10.589474  ,
        -10.808492  ,  -0.24790464],
       [  0.9988323 ,  -0.32367784, -21.522228  , ...,   4.6680403 ,
          6.1433697 ,  -0.23947598]], dtype=float32)

In [10]:
vgg16_train_features.shape

(3210, 4096)

In [11]:
vgg16_test_features[:5]

array([[ -3.9410076 ,  -0.6707838 , -53.463104  , ...,  -8.278335  ,
        -10.292274  ,  -0.265556  ],
       [ -0.1634852 ,  -0.5534688 , -22.074677  , ...,   5.6482077 ,
         18.613846  ,  -0.42996874],
       [  1.8922055 ,  -0.8144202 , -27.296654  , ...,   8.652847  ,
          0.21162796,  -0.26159078],
       [ -1.96607   ,  -0.30473575,  -4.8362856 , ...,   2.0165455 ,
         -1.1177328 ,  -0.26760662],
       [ -1.2365555 ,  -0.18508121,   9.221886  , ...,   0.6952783 ,
         -2.750974  ,  -0.17681438]], dtype=float32)

In [12]:
# Load vgg16 features after relu: should not contain negative value
vgg16_relu_train_features = np.load('vgg16_relu_train_features.npy')
vgg16_relu_test_features = np.load('vgg16_relu_test_features.npy')

In [13]:
vgg16_relu_train_features[:5]

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.9988323, 0.       , 0.       , ..., 4.6680403, 6.1433697,
        0.       ]], dtype=float32)

In [14]:
vgg16_relu_train_features.shape

(3210, 4096)

In [15]:
vgg16_relu_test_features[:5]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  5.6482077 ,
        18.613846  ,  0.        ],
       [ 1.8922055 ,  0.        ,  0.        , ...,  8.652847  ,
         0.21162796,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  2.0165455 ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  9.221886  , ...,  0.6952783 ,
         0.        ,  0.        ]], dtype=float32)

In [16]:
# Load resnet features
resnet_train_features = np.load('resnet_train_features.npy')
resnet_test_features = np.load('resnet_test_features.npy')

In [17]:
resnet_train_features[:5] 

array([[[[0.07640424, 0.11314762, 0.7201315 , ..., 7.5590777 ,
          2.6926625 , 0.        ]]],


       [[[0.33827135, 1.4292572 , 0.        , ..., 5.61894   ,
          0.        , 0.        ]]],


       [[[0.01324131, 6.6258082 , 0.        , ..., 0.        ,
          0.        , 7.274989  ]]],


       [[[0.12476952, 2.2230198 , 4.2002983 , ..., 0.56822747,
          0.        , 0.20838165]]],


       [[[0.7693701 , 0.        , 0.26286367, ..., 1.2396903 ,
          1.1977144 , 0.        ]]]], dtype=float32)

In [18]:
resnet_train_features.shape

(3210, 1, 1, 2048)

In [19]:
# Flatten the features for SVR
resnet_train_features_flatten = resnet_train_features.reshape(resnet_train_features.shape[0], -1)
resnet_test_features_flatten = resnet_test_features.reshape(resnet_test_features.shape[0], -1)

In [20]:
resnet_train_features_flatten.shape

(3210, 2048)

In [21]:
# Load senet features
senet_train_features = np.load('senet_train_features.npy')
senet_test_features = np.load('senet_test_features.npy')

In [22]:
senet_train_features[:5]

array([[[[1.77558716e-02, 3.35661054e-01, 1.13176536e-02, ...,
          1.72436726e+00, 1.63295358e-01, 5.43274581e-02]]],


       [[[4.50852998e-02, 4.48929369e-02, 1.28231063e-01, ...,
          3.73613667e+00, 1.59444585e-02, 4.65025380e-02]]],


       [[[1.77555485e+01, 3.79963303e+00, 4.00157642e+00, ...,
          1.12641087e+01, 1.84112501e+00, 5.62081598e-02]]],


       [[[1.01859818e+01, 7.96744967e+00, 1.34816945e+00, ...,
          3.84514546e+00, 1.06552868e+01, 5.48046865e-02]]],


       [[[5.14905304e-02, 3.44009288e-02, 1.63564754e+00, ...,
          2.31483225e-02, 1.34842947e-01, 1.02118626e-01]]]],
      dtype=float32)

In [23]:
# Flatten the features for SVR
senet_train_features_flatten = senet_train_features.reshape(senet_train_features.shape[0], -1)
senet_test_features_flatten = senet_test_features.reshape(senet_test_features.shape[0], -1)

## SVR with Hyperparameter Tuning

### UDF

In [24]:
def evaluate_performance(true_values, predicted_values):
    rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
    mae = mean_absolute_error(true_values, predicted_values)
    corr = np.corrcoef(true_values, predicted_values)[0][1]
    
    print("RMSE: ", rmse)
    print("MAE: ", mae)
    print("Pearson Correlation: ", corr)

In [25]:
# Define a custom scorer
def pearson_corr(y_true, y_pred):
    corr = np.corrcoef(y_true, y_pred)[0][1]
    return corr

In [26]:
# Create a scorer from the custom scorer function
pearson_scorer = make_scorer(pearson_corr)

### VGG16 fc6

#### Linear

In [27]:
"""
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10]
}

# Create a SVR model
svr_linear = SVR(kernel='linear')

# Create the grid search object
grid_search_SVR_linear = GridSearchCV(estimator=svr_linear, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_linear.fit(vgg16_train_features, train_label)

print(grid_search_SVR_linear.best_params_)
print(grid_search_SVR_linear.best_estimator_)
vgg16_SVR_linear = grid_search_SVR_linear.best_estimator_

# Use the trained model to predict BMI on the test data
vgg16_linear_predictions = vgg16_SVR_linear.predict(vgg16_test_features)

evaluate_performance(test_label, vgg16_linear_predictions)
"""

"\n# Define the parameter grid\nparam_grid = {\n    'C': [0.1, 1, 10],\n    'epsilon': [0.01, 0.1, 10]\n}\n\n# Create a SVR model\nsvr_linear = SVR(kernel='linear')\n\n# Create the grid search object\ngrid_search_SVR_linear = GridSearchCV(estimator=svr_linear, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)\n\n# Perform grid search\ngrid_search_SVR_linear.fit(vgg16_train_features, train_label)\n\nprint(grid_search_SVR_linear.best_params_)\nprint(grid_search_SVR_linear.best_estimator_)\nvgg16_SVR_linear = grid_search_SVR_linear.best_estimator_\n\n# Use the trained model to predict BMI on the test data\nvgg16_linear_predictions = vgg16_SVR_linear.predict(vgg16_test_features)\n\nevaluate_performance(test_label, vgg16_linear_predictions)\n"

#### Polynomial

In [28]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10],
    'degree': [2, 3, 4]
}

# Create a SVR model
svr_poly = SVR(kernel='poly')

# Create the grid search object
grid_search_SVR_poly = GridSearchCV(estimator=svr_poly, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=2)

# Perform grid search
grid_search_SVR_poly.fit(vgg16_train_features, train_label)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


GridSearchCV(cv=5, estimator=SVR(kernel='poly'), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'degree': [2, 3, 4],
                         'epsilon': [0.01, 0.1, 10]},
             scoring=make_scorer(pearson_corr), verbose=2)

In [29]:
print(grid_search_SVR_poly.best_params_)
print(grid_search_SVR_poly.best_estimator_)
vgg16_SVR_poly = grid_search_SVR_poly.best_estimator_

{'C': 10, 'degree': 2, 'epsilon': 0.1}
SVR(C=10, degree=2, kernel='poly')


In [30]:
# Use the trained model to predict BMI on the test data
vgg16_poly_predictions = vgg16_SVR_poly.predict(vgg16_test_features)

In [31]:
evaluate_performance(test_label, vgg16_poly_predictions)

RMSE:  7.603566619485435
MAE:  5.387772354730961
Pearson Correlation:  0.5884489544571193


#### RBF

In [32]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10],
}

# Create a SVR model
svr_rbf = SVR(kernel='rbf')

# Create the grid search object
grid_search_SVR_rbf = GridSearchCV(estimator=svr_rbf, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_rbf.fit(vgg16_train_features, train_label)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 10]},
             scoring=make_scorer(pearson_corr))

In [33]:
print(grid_search_SVR_rbf.best_params_)
print(grid_search_SVR_rbf.best_estimator_)
vgg16_SVR_rbf = grid_search_SVR_rbf.best_estimator_

{'C': 10, 'epsilon': 0.1}
SVR(C=10)


In [34]:
# Use the trained model to predict BMI on the test data
vgg16_rbf_predictions = vgg16_SVR_rbf.predict(vgg16_test_features)

In [35]:
evaluate_performance(test_label, vgg16_rbf_predictions)

RMSE:  7.1332604248244325
MAE:  5.002939488572106
Pearson Correlation:  0.648734532100628


In [36]:
import joblib

# Save the trained SVR model
joblib.dump(vgg16_SVR_rbf, 'vgg16_SVR_rbf_model.pkl')

['vgg16_SVR_rbf_model.pkl']

### VGG16 fc6 relu

#### Linear

In [37]:
"""
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10]
}

# Create a SVR model
svr_linear = SVR(kernel='linear')

# Create the grid search object
grid_search_SVR_linear = GridSearchCV(estimator=svr_linear, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_linear.fit(vgg16_relu_train_features, train_label)

print(grid_search_SVR_linear.best_params_)
print(grid_search_SVR_linear.best_estimator_)
vgg16_relu_SVR_linear = grid_search_SVR_linear.best_estimator_

# Use the trained model to predict BMI on the test data
vgg16_relu_linear_predictions = vgg16_relu_SVR_linear.predict(vgg16_relu_test_features)

evaluate_performance(test_label, vgg16_relu_linear_predictions)
"""

"\n# Define the parameter grid\nparam_grid = {\n    'C': [0.1, 1, 10],\n    'epsilon': [0.01, 0.1, 10]\n}\n\n# Create a SVR model\nsvr_linear = SVR(kernel='linear')\n\n# Create the grid search object\ngrid_search_SVR_linear = GridSearchCV(estimator=svr_linear, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)\n\n# Perform grid search\ngrid_search_SVR_linear.fit(vgg16_relu_train_features, train_label)\n\nprint(grid_search_SVR_linear.best_params_)\nprint(grid_search_SVR_linear.best_estimator_)\nvgg16_relu_SVR_linear = grid_search_SVR_linear.best_estimator_\n\n# Use the trained model to predict BMI on the test data\nvgg16_relu_linear_predictions = vgg16_relu_SVR_linear.predict(vgg16_relu_test_features)\n\nevaluate_performance(test_label, vgg16_relu_linear_predictions)\n"

#### Polynoimial

In [38]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10],
    'degree': [2, 3, 4]
}

# Create a SVR model
svr_poly = SVR(kernel='poly')

# Create the grid search object
grid_search_SVR_poly = GridSearchCV(estimator=svr_poly, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=2)

# Perform grid search
grid_search_SVR_poly.fit(vgg16_relu_train_features, train_label)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5, estimator=SVR(kernel='poly'), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'degree': [2, 3, 4],
                         'epsilon': [0.01, 0.1, 10]},
             scoring=make_scorer(pearson_corr), verbose=2)

In [39]:
print(grid_search_SVR_poly.best_params_)
print(grid_search_SVR_poly.best_estimator_)
vgg16_relu_SVR_poly = grid_search_SVR_poly.best_estimator_

{'C': 10, 'degree': 2, 'epsilon': 0.1}
SVR(C=10, degree=2, kernel='poly')


In [40]:
# Use the trained model to predict BMI on the test data
vgg16_relu_poly_predictions = vgg16_relu_SVR_poly.predict(vgg16_relu_test_features)

In [41]:
evaluate_performance(test_label, vgg16_relu_poly_predictions)

RMSE:  7.62557256123413
MAE:  5.333885969951235
Pearson Correlation:  0.5831755720281216


#### RBF

In [42]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10]
}

# Create a SVR model
svr_rbf = SVR(kernel='rbf')

# Create the grid search object
grid_search_SVR_rbf = GridSearchCV(estimator=svr_rbf, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_rbf.fit(vgg16_relu_train_features, train_label)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 10]},
             scoring=make_scorer(pearson_corr))

In [43]:
print(grid_search_SVR_rbf.best_params_)
print(grid_search_SVR_rbf.best_estimator_)
vgg16_relu_SVR_rbf = grid_search_SVR_rbf.best_estimator_

{'C': 10, 'epsilon': 0.1}
SVR(C=10)


In [44]:
# Use the trained model to predict BMI on the test data
vgg16_relu_rbf_predictions = vgg16_relu_SVR_rbf.predict(vgg16_relu_test_features)

In [45]:
evaluate_performance(test_label, vgg16_relu_rbf_predictions)

RMSE:  7.124819107776734
MAE:  4.971186911512504
Pearson Correlation:  0.6494042625598317


In [46]:
joblib.dump(vgg16_relu_SVR_rbf, 'vgg16_relu_SVR_rbf_model.pkl')

['vgg16_relu_SVR_rbf_model.pkl']

### Test

#### Example

In [ ]:
%%time
test_svr = SVR(kernel = 'rbf', epsilon = 0.1)
test_svr_model = test_svr.fit(vgg16_train_features, train_label)
test_predictions = test_svr_model.predict(vgg16_test_features)
print(np.corrcoef(test_label, test_predictions))

In [ ]:
%%time
test_svr = SVR(kernel = 'linear', epsilon = 0.1)
test_svr_model = test_svr.fit(vgg16_train_features, train_label)
test_predictions = test_svr_model.predict(vgg16_test_features)
print(np.corrcoef(test_label, test_predictions))

### VGG Resnet

#### Linear

In [ ]:
"""
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10]
}

# Create a SVR model
svr_linear = SVR(kernel='linear')

# Create the grid search object
grid_search_SVR_linear = GridSearchCV(estimator=svr_linear, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_linear.fit(resnet_train_features_flatten, train_label)

print(grid_search_SVR_linear.best_params_)
print(grid_search_SVR_linear.best_estimator_)
resnet_SVR_linear = grid_search_SVR_linear.best_estimator_

# Use the trained model to predict BMI on the test data
resnet_linear_predictions = resnet_SVR_linear.predict(resnet_test_features_flatten)

evaluate_performance(test_label, resnet_linear_predictions)
"""

#### Polynomial

In [48]:
%%time
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10],
    'degree': [2, 3, 4]
}

# Create a SVR model
svr_poly = SVR(kernel='poly')

# Create the grid search object
grid_search_SVR_poly = GridSearchCV(estimator=svr_poly, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_poly.fit(resnet_train_features_flatten, train_label)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


CPU times: user 15 s, sys: 418 ms, total: 15.4 s
Wall time: 3min 23s


GridSearchCV(cv=5, estimator=SVR(kernel='poly'), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'degree': [2, 3, 4],
                         'epsilon': [0.01, 0.1, 10]},
             scoring=make_scorer(pearson_corr))

In [49]:
print(grid_search_SVR_poly.best_params_)
print(grid_search_SVR_poly.best_estimator_)
resnet_SVR_poly = grid_search_SVR_poly.best_estimator_

{'C': 1, 'degree': 2, 'epsilon': 0.1}
SVR(C=1, degree=2, kernel='poly')


In [50]:
# Use the trained model to predict BMI on the test data
resnet_poly_predictions = resnet_SVR_poly.predict(resnet_test_features_flatten)

In [51]:
evaluate_performance(test_label, resnet_poly_predictions)

RMSE:  7.351026962244068
MAE:  5.12953008294609
Pearson Correlation:  0.6344168524038006


#### RBF

In [52]:
%%time
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10]
}

# Create a SVR model
svr_rbf = SVR(kernel='rbf')

# Create the grid search object
grid_search_SVR_rbf = GridSearchCV(estimator=svr_rbf, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_rbf.fit(resnet_train_features_flatten, train_label)

CPU times: user 14.7 s, sys: 63.7 ms, total: 14.8 s
Wall time: 1min 19s


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 10]},
             scoring=make_scorer(pearson_corr))

In [53]:
print(grid_search_SVR_rbf.best_params_)
print(grid_search_SVR_rbf.best_estimator_)
resnet_SVR_rbf = grid_search_SVR_rbf.best_estimator_

{'C': 10, 'epsilon': 0.1}
SVR(C=10)


In [54]:
# Use the trained model to predict BMI on the test data
resnet_rbf_predictions = resnet_SVR_rbf.predict(resnet_test_features_flatten)

In [55]:
evaluate_performance(test_label, resnet_rbf_predictions)

RMSE:  7.062309705808011
MAE:  5.020718282284416
Pearson Correlation:  0.6563192803613689


In [56]:
joblib.dump(resnet_SVR_rbf, 'resnet_SVR_rbf_model.pkl')

['resnet_SVR_rbf_model.pkl']

### VGG Senet

#### Polynomial

In [57]:
%%time
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10],
    'degree': [2, 3, 4]
}

# Create a SVR model
svr_poly = SVR(kernel='poly')

# Create the grid search object
grid_search_SVR_poly = GridSearchCV(estimator=svr_poly, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_poly.fit(senet_train_features_flatten, train_label)

CPU times: user 14.3 s, sys: 108 ms, total: 14.4 s
Wall time: 3min 11s


GridSearchCV(cv=5, estimator=SVR(kernel='poly'), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'degree': [2, 3, 4],
                         'epsilon': [0.01, 0.1, 10]},
             scoring=make_scorer(pearson_corr))

In [58]:
print(grid_search_SVR_poly.best_params_)
print(grid_search_SVR_poly.best_estimator_)
senet_SVR_poly = grid_search_SVR_poly.best_estimator_

{'C': 1, 'degree': 2, 'epsilon': 0.1}
SVR(C=1, degree=2, kernel='poly')


In [59]:
# Use the trained model to predict BMI on the test data
senet_poly_predictions = senet_SVR_poly.predict(senet_test_features_flatten)

In [60]:
evaluate_performance(test_label, senet_poly_predictions)

RMSE:  7.1501955597911016
MAE:  4.961102982743951
Pearson Correlation:  0.661685765107273


In [61]:
joblib.dump(senet_SVR_poly, 'senet_SVR_poly_model.pkl')

['senet_SVR_poly_model.pkl']

#### RBF

In [62]:
%%time
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 10]
}

# Create a SVR model
svr_rbf = SVR(kernel='rbf')

# Create the grid search object
grid_search_SVR_rbf = GridSearchCV(estimator=svr_rbf, param_grid=param_grid, cv=5, scoring=pearson_scorer, n_jobs=-1, verbose=0)

# Perform grid search
grid_search_SVR_rbf.fit(senet_train_features_flatten, train_label)

CPU times: user 14.6 s, sys: 67.6 ms, total: 14.7 s
Wall time: 1min 16s


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 10]},
             scoring=make_scorer(pearson_corr))

In [63]:
print(grid_search_SVR_rbf.best_params_)
print(grid_search_SVR_rbf.best_estimator_)
senet_SVR_rbf = grid_search_SVR_rbf.best_estimator_

{'C': 10, 'epsilon': 0.1}
SVR(C=10)


In [64]:
# Use the trained model to predict BMI on the test data
senet_rbf_predictions = senet_SVR_rbf.predict(senet_test_features_flatten)

In [65]:
evaluate_performance(test_label, senet_rbf_predictions)

RMSE:  6.961167049497936
MAE:  4.838214594500182
Pearson Correlation:  0.6681268285111877


In [66]:
joblib.dump(senet_SVR_rbf, 'senet_SVR_rbf_model.pkl')

['senet_SVR_rbf_model.pkl']

## Conclusion

Both RESNET50 and SENET50 leads to the featuers that makes higher correlation combined with SVR with RBF kernel